In [1]:
import psycopg2
import os
import sqlalchemy
import pandas as pd
import pandas.io.sql as psql
import sys
from functools import lru_cache

import boto3
import os

In [2]:
def aws_session(aws_profile=None):
    """Create a a boto3 session.
    Params:
        (string): credentials profile name
    Returns:
        (boto3 client object)
    """
    session = boto3.Session()
    # If the session is run on a local machine, with AWS credentials fetched
    # from a shared file, use the DataScience role profile.
    if session.get_credentials().method == 'shared-credentials-file':
        session = boto3.Session(profile_name=aws_profile)
    creds = session.get_credentials()
    result = {
        'aws_access_key_id': creds.access_key,
        'aws_secret_access_key': creds.secret_key,
        'aws_session_token': creds.token
    }
    return result

CREDS = aws_session('sg_stage')

@lru_cache()
def s3_client():
    """Cache a boto3 client with credentias and MFA token."""
    return boto3.client(
            service_name='s3',
            aws_access_key_id=CREDS['aws_access_key_id'],
            aws_secret_access_key=CREDS['aws_secret_access_key'],
            aws_session_token=CREDS['aws_session_token'],
    )

Enter MFA code for arn:aws:iam::049635306943:mfa/jing.xin:  ······


In [3]:
sample_file = 'testsample_farmpulse.csv'
bucket = 'sh-farmpulse-stage'
sample_key = f'FarmPulse Cleaned csv (no kcc_raw)/{sample_file}'

### Download sample from S3 bucket

In [16]:
try:
    s3 = s3_client()
    s3.download_file(bucket, sample_key, sample_file)
    print("Download Successful")
except Exception as e:
    print(e)

Download Successful


### Convert raw Farmpulse data (India table) to dataframe
Note that this is after restoring farmpulse dump file on local SQL database

In [4]:
# connect to SQL database to retrieve farmpulse data
try:
    connection = psycopg2.connect(user="postgres",
                                  host="localhost",
                                  port="5432",
                                  database="farmpulse")

    cursor = connection.cursor()

except (Exception, psycopg2.Error) as error :
    print("Error while connecting to PostgreSQL", error)

In [5]:
postgreSQL_select_Query = "select * from call_centre.india"

cursor.execute(postgreSQL_select_Query)
records = cursor.fetchall()

In [6]:
num_fields = len(cursor.description)
field_names = [i[0] for i in cursor.description]
df = pd.DataFrame(records, columns=field_names)
df.head(5)

,created_on,state_name,district_name,block_name,latitude,longitude,season,sector,category,crop,query_type,query_text_raw,query_text_translate,query_text_hindi,query_text_english,kcc_answer_raw,kcc_answer_translate,kcc_answer_hindi,kcc_answer_english,id
0,2014-12-03 10:25:20,rajasthan,ganganagar,anupgarh,29.181153,73.199860,rabi,agriculture,pulses,bengal gram (gram/chick pea/kabuli/chana),plant protection,TELL ME WEED CONTROL OF BENGAL GRAM,False,None,TELL ME WEED CONTROL OF BENGAL GRAM,YOU CAN NOT USE HERBICIDE IN STANDING CROP OF ...,False,None,YOU CAN NOT USE HERBICIDE IN STANDING CROP OF ...,9883396
1,2014-12-03 10:27:50,rajasthan,chittaurgarh,nimbahera,24.625659,74.681027,rabi,agriculture,pulses,bengal gram (gram/chick pea/kabuli/chana),plant protection,TELL ME CONTROL OF BORER IN GRAM ?,False,None,TELL ME CONTROL OF BORER IN GRAM ?,SPRAY OF PROFENOFOS 1.5 ML PER LITER WATER,False,None,SPRAY OF PROFENOFOS 1.5 ML PER LITER WATER,9883397
2,2014-12-03 10:28:18,rajasthan,hanumangarh,nohar,29.174841,74.768817,rabi,agriculture,others,others,government schemes,TELL ME SOLAR SBSIDY,False,None,TELL ME SOLAR SBSIDY,70 % SUBSIDY OF TOTAL COST ON SOLAR PLANT,False,None,70 % SUBSIDY OF TOTAL COST ON SOLAR PLANT,9883398
3,2014-12-03 10:28:20,rajasthan,bharatpur,kumher,27.342644,77.372638,rabi,agriculture,oilseeds,mustard,plant protection,TELL ME INSECT CONTROL IN MUSTRAD,False,None,TELL ME INSECT CONTROL IN MUSTRAD,SPRAY OF IMIDACHLOPRID(CONFIDOR) 17.8% SL 1 LI...,False,None,SPRAY OF IMIDACHLOPRID(CONFIDOR) 17.8% SL 1 LI...,9883399
4,2014-12-03 10:28:51,rajasthan,nagaur,merta,26.644952,74.030942,rabi,agriculture,cereals,wheat,plant protection,seed treatment of wheat,False,None,seed treatment of wheat,USE CARBENDAZIM 2 GM PER KG SEED,False,None,USE CARBENDAZIM 2 GM PER KG SEED,9883400


In [11]:
df.to_csv('india_table.csv', sep='\t')

### Generate size of full Farmpulse dataframe

In [8]:
import time

start_time = time.time()
total_size = sys.getsizeof(df)

print(f'total size = {total_size} bytes, {total_size/(1024*1024*1024)}GB')
print("took", time.time() - start_time, "to run")

total size = 22750229424 bytes, 21.187802240252495GB
took 429.5599453449249 to run


### Generate and Upload small sample of Farmpulse data

In [9]:
sample = pd.read_sql('select * from call_centre.india limit 100',
                     con=connection)
sample.head(5)

,created_on,state_name,district_name,block_name,latitude,longitude,season,sector,category,crop,query_type,query_text_raw,query_text_translate,query_text_hindi,query_text_english,kcc_answer_raw,kcc_answer_translate,kcc_answer_hindi,kcc_answer_english,id
0,2014-12-03 10:25:20,rajasthan,ganganagar,anupgarh,29.181153,73.199860,rabi,agriculture,pulses,bengal gram (gram/chick pea/kabuli/chana),plant protection,TELL ME WEED CONTROL OF BENGAL GRAM,False,None,TELL ME WEED CONTROL OF BENGAL GRAM,YOU CAN NOT USE HERBICIDE IN STANDING CROP OF ...,False,None,YOU CAN NOT USE HERBICIDE IN STANDING CROP OF ...,9883396
1,2014-12-03 10:27:50,rajasthan,chittaurgarh,nimbahera,24.625659,74.681027,rabi,agriculture,pulses,bengal gram (gram/chick pea/kabuli/chana),plant protection,TELL ME CONTROL OF BORER IN GRAM ?,False,None,TELL ME CONTROL OF BORER IN GRAM ?,SPRAY OF PROFENOFOS 1.5 ML PER LITER WATER,False,None,SPRAY OF PROFENOFOS 1.5 ML PER LITER WATER,9883397
2,2014-12-03 10:28:18,rajasthan,hanumangarh,nohar,29.174841,74.768817,rabi,agriculture,others,others,government schemes,TELL ME SOLAR SBSIDY,False,None,TELL ME SOLAR SBSIDY,70 % SUBSIDY OF TOTAL COST ON SOLAR PLANT,False,None,70 % SUBSIDY OF TOTAL COST ON SOLAR PLANT,9883398
3,2014-12-03 10:28:20,rajasthan,bharatpur,kumher,27.342644,77.372638,rabi,agriculture,oilseeds,mustard,plant protection,TELL ME INSECT CONTROL IN MUSTRAD,False,None,TELL ME INSECT CONTROL IN MUSTRAD,SPRAY OF IMIDACHLOPRID(CONFIDOR) 17.8% SL 1 LI...,False,None,SPRAY OF IMIDACHLOPRID(CONFIDOR) 17.8% SL 1 LI...,9883399
4,2014-12-03 10:28:51,rajasthan,nagaur,merta,26.644952,74.030942,rabi,agriculture,cereals,wheat,plant protection,seed treatment of wheat,False,None,seed treatment of wheat,USE CARBENDAZIM 2 GM PER KG SEED,False,None,USE CARBENDAZIM 2 GM PER KG SEED,9883400


In [10]:
sample.to_csv(sample_file, sep='\t')

In [14]:
try:
    s3 = s3_client()
    s3.upload_file(sample_file, bucket, sample_key)
    print("Upload Successful")
except FileNotFoundError:
    print("The file was not found")

Upload Successful
